In [26]:
import rasterio
import numpy as np
import geopandas as gpd
import io
import base64
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
from rasterio.transform import rowcol
from ipyleaflet import (
    Map, GeoJSON, FullScreenControl, ScaleControl, ImageOverlay,
    DrawControl, WidgetControl, Marker, LayersControl, TileLayer
)
from ipywidgets import HTML, FloatSlider, Label
from ipyleaflet import (
    Map, DrawControl, Marker, Popup, ImageOverlay, TileLayer, LayersControl, 
    FullScreenControl, ZoomControl, ScaleControl, WidgetControl
)
from ipywidgets import HTML, VBox, Textarea, Dropdown, Label, Text, Button, Layout
from IPython.display import display  # Asegura que el mapa se renderice correctamente

# ----------------------------------------------
# 1. Función genérica para reproyectar ráster
# ----------------------------------------------
def reproject_raster(src_path, discrete=False):
    """
    Reproyecta cualquier ráster a EPSG:4326.
    
    Parámetros:
    -----------
    src_path: str
        Path al archivo ráster.
    discrete: bool
        Si es True, se usa 'nearest' para resampling (por ej. capas categóricas).
        Si es False, se usa 'bilinear' (para datos continuos).
    
    Retorna:
    --------
    (bounds, transform, raster_array)
        - bounds: (left, bottom, right, top) en EPSG:4326
        - transform: objeto Affine del ráster reproyectado
        - raster_array: numpy array con valores reproyectados y nodata=NaN
    """
    with rasterio.open(src_path) as src:
        # Elegir Resampling
        resamp_method = Resampling.nearest if discrete else Resampling.bilinear
        
        # Reproyectar si no está en EPSG:4326
        if src.crs is None or src.crs.to_string() != "EPSG:4326":
            transform, width, height = calculate_default_transform(
                src.crs, "EPSG:4326", src.width, src.height, *src.bounds
            )
            data = np.empty((height, width), dtype=np.float32)

            reproject(
                source=rasterio.band(src, 1),
                destination=data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs="EPSG:4326",
                resampling=resamp_method
            )
        else:
            data = src.read(1)      # Band 1
            transform = src.transform

        # Leer nodata si existe
        nodata_value = src.nodata

        # Crear máscara de nodata
        if nodata_value is not None:
            mask_nodata = (data == nodata_value)
        else:
            mask_nodata = np.zeros_like(data, dtype=bool)

        # Filtrar valores aberrantes (por ejemplo > 1e10)
        mask_aberrantes = (data > 1e10)

        # Unir ambas máscaras
        final_mask = np.logical_or(mask_nodata, mask_aberrantes)

        # Asignar NaN donde hay nodata o aberrantes
        data = np.where(final_mask, np.nan, data)

        # Calcular límites en EPSG:4326
        bounds = rasterio.warp.transform_bounds(src.crs, "EPSG:4326", *src.bounds)

    return bounds, transform, data

# ----------------------------------------------
# 2. Convertir array ráster a imagen PNG Base64
# ----------------------------------------------
def raster_to_base64(raster_array, bounds, cmap="terrain"):
    """
    Dibuja el array ráster y lo devuelve como imagen PNG codificada en Base64.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    left, bottom, right, top = bounds

    ax.imshow(raster_array, cmap=cmap, extent=[left, right, bottom, top], origin="upper")
    ax.axis("off")

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', transparent=True)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode()
    buf.close()
    plt.close(fig)
    return img_base64

# ----------------------------------------------
# 3. Crear mapa con múltiples capas ráster
# ----------------------------------------------
def create_interactive_map(
    vector_path,
    river_network_path,
    # Lista de rásteres a cargar con su configuración
    rasters_config
):
    """
    Crea un mapa ipyleaflet con capas vectoriales, múltiples rásteres, 
    un slider de opacidad único, y popups/widgets mostrando valores.
    
    Parámetros:
    -----------
    vector_path: str
        GeoJSON o shapefile de la cuenca
    river_network_path: str
        GeoJSON o shapefile de la red de drenaje
    rasters_config: list of dict
        Cada dict debe contener:
         {
           "name": str,                # nombre de la capa
           "path": str,                # ruta al .tif
           "discrete": bool,           # True -> nearest, False -> bilinear
           "colormap": str (opcional)  # colormap de matplotlib
         }
    """
    # 3.1 Cargar capa vectorial (cuenca)
    gdf = gpd.read_file(vector_path)
    if gdf.crs is None or gdf.crs.to_string() != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # 3.2 Cargar capa de drenaje
    river_gdf = gpd.read_file(river_network_path)
    if river_gdf.crs is None or river_gdf.crs.to_string() != "EPSG:4326":
        river_gdf = river_gdf.to_crs("EPSG:4326")

    # Calcular un centro de mapa
    centroid = gdf.geometry.unary_union.centroid
    center_y, center_x = centroid.y, centroid.x

    # 3.3 Crear el mapa
    the_map = Map(center=(center_y, center_x), zoom=10, scroll_wheel_zoom=True)
    the_map.add_control(FullScreenControl())
    the_map.add_control(ScaleControl(position="bottomleft"))
    
    # Establecer altura inicial para el toggle
    the_map.layout = Layout(height='700px')

    # 3.4 Agregar cuenca y drenaje
    geojson_layer = GeoJSON(
        data=gdf.__geo_interface__,
        name="Cuenca Vectorial",
        style={'color': 'blue', 'weight': 2, 'fillOpacity': 0.4}
    )
    the_map.add_layer(geojson_layer)

    river_layer = GeoJSON(
        data=river_gdf.__geo_interface__,
        name="Red de Drenaje",
        style={'color': 'cyan', 'weight': 1, 'fillOpacity': 0.7}
    )
    the_map.add_layer(river_layer)

    # -----------------------------------------
    # 3.5 Procesar y añadir todas las capas ráster
    # -----------------------------------------
    all_rasters = {}
    all_overlays = []

    # Para ajustar los límites del mapa con la primera capa
    first_bounds = None

    for cfg in rasters_config:
        r_name = cfg["name"]
        r_path = cfg["path"]
        r_discrete = cfg.get("discrete", False)
        r_cmap = cfg.get("colormap", "terrain")  # por defecto "terrain"

        # Reproyectar
        bounds, transform, arr = reproject_raster(r_path, discrete=r_discrete)

        # Convertir a imagen Base64
        img_b64 = raster_to_base64(arr, bounds, cmap=r_cmap)

        # Crear overlay
        overlay = ImageOverlay(
            url=f"data:image/png;base64,{img_b64}",
            bounds=((bounds[1], bounds[0]), (bounds[3], bounds[2])),
            name=r_name,
            opacity=0.6
        )
        the_map.add_layer(overlay)
        all_overlays.append(overlay)

        # Guardar info para posterior consulta
        all_rasters[r_name] = {
            "array": arr,
            "transform": transform,
            "bounds": bounds,
            "discrete": r_discrete
        }

        # Actualizar first_bounds si es la primera
        if first_bounds is None:
            first_bounds = bounds

    # Ajustar límites del mapa usando la primera capa de la lista
    if first_bounds is not None:
        the_map.fit_bounds(((first_bounds[1], first_bounds[0]),
                            (first_bounds[3], first_bounds[2])))

    # -----------------------------------------
    # 3.6 Único slider para opacidad de TODAS las capas
    # -----------------------------------------
    opacity_slider = FloatSlider(
        value=0.6, min=0, max=1, step=0.05, description="Opacidad Ráster"
    )

    def update_opacity(change):
        new_op = change['new']
        for ov in all_overlays:
            ov.opacity = new_op

    opacity_slider.observe(update_opacity, 'value')
    the_map.add_control(WidgetControl(widget=opacity_slider, position="bottomleft"))

    # Crear botón para alternar tamaño
    toggle_button = Button(description="Ampliar Mapa")

    # Función para cambiar tamaño
    def toggle_size(b):
        if the_map.layout.height == '700px':
            the_map.layout.height = '1000px'  # Tamaño grande
            toggle_button.description = "Reducir Mapa"
        else:
            the_map.layout.height = '700px'   # Tamaño normal
            toggle_button.description = "Ampliar Mapa"

    toggle_button.on_click(toggle_size)

    # Agregar control del botón en el mapa
    the_map.add_control(WidgetControl(widget=toggle_button, position="topright"))

    # -----------------------------------------
    # 3.7 Añadir capas base y control de capas
    # -----------------------------------------
    hillshade = TileLayer(
        url="https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}",
        name="Hillshade (Esri)",
        attribution="Esri"
    )

    carto_positron = TileLayer(
        url="https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png",
        name="Carto Positron",
        attribution="CartoDB"
    )

    esri_sat = TileLayer(
        url="https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        name="Satélite (Esri)",
        attribution="Esri"
    )

    esri_topo = TileLayer(
        url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
        name="Esri Topográfico",
        attribution="Esri"
    )

    esri_terrain = TileLayer(
        url="https://server.arcgisonline.com/arcgis/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}",
        name="Esri Terrain",
        attribution="Esri"
    )

    osm_backup = TileLayer(
        url="https://b.tile.openstreetmap.fr/hot/{z}/{x}/{y}.png",
        name="OpenStreetMap Backup",
        attribution="OpenStreetMap"
    )

    google_hybrid = TileLayer(
        url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        name="Google Hybrid",
        attribution="Google"
    )

    the_map.add_layer(osm_backup)
    the_map.add_layer(google_hybrid)
    the_map.add_layer(esri_sat)
    the_map.add_layer(esri_terrain)
    the_map.add_layer(esri_topo)
    the_map.add_layer(carto_positron)
    the_map.add_layer(hillshade)

    the_map.add_control(LayersControl(position="topright"))

    # -----------------------------------------
    # 3.8 Funciones para obtener valor de cada ráster en (lat, lon)
    # -----------------------------------------
    def get_raster_value(raster_name, lat, lon):
        """
        Retorna el valor del ráster con nombre 'raster_name' en la coordenada (lat, lon).
        """
        info = all_rasters[raster_name]
        arr = info["array"]
        transform = info["transform"]

        try:
            row, col = rowcol(transform, lon, lat)
            nrow, ncol = arr.shape
            if 0 <= row < nrow and 0 <= col < ncol:
                val = arr[row, col]
                if np.isnan(val):
                    return "No disponible"
                # Si es un ráster discreto, lo mostramos como entero
                if info["discrete"]:
                    return int(val)
                else:
                    return round(float(val), 2)
            else:
                return "No disponible"
        except:
            return "No disponible"

    # -----------------------------------------
    # 3.9 DrawControl: Añadir marcadores que muestren info de TODAS las capas
    # -----------------------------------------
    draw_control = DrawControl()
    the_map.add_control(draw_control)

    def handle_draw(target, action, geo_json):
        if action == "created" and geo_json['geometry']['type'] == "Point":
            lon, lat = geo_json['geometry']['coordinates']

            # Armar la info: iteramos sobre todos los rásteres
            popup_lines = [
                f"<b>{rname}:</b> {get_raster_value(rname, lat, lon)}"
                for rname in all_rasters.keys()
            ]
            popup_text = (
                f"<b>Lat:</b> {lat:.6f}<br>"
                f"<b>Lon:</b> {lon:.6f}<br>"
                + "<br>".join(popup_lines)
            )

            marker = Marker(location=(lat, lon))
            marker.popup = HTML(value=popup_text)
            the_map.add_layer(marker)

    draw_control.on_draw(handle_draw)

    # -----------------------------------------
    # 3.10 Widget de coordenadas dinámicas
    # -----------------------------------------
    # Inicialmente, mostramos "Lat: --, Lon: --" + nombres de ráster con "--"
    label_text = "Lat: --, Lon: --, " + ", ".join([f"{rname}: --" for rname in all_rasters.keys()])
    coordinates_label = Label(value=label_text)
    coordinates_control = WidgetControl(widget=coordinates_label, position="bottomright")
    the_map.add_control(coordinates_control)

    def update_coordinates(event, **kwargs):
        if 'coordinates' in kwargs:
            lat, lon = kwargs['coordinates']
            # Para cada ráster, obtenemos el valor
            raster_vals = [f"{rn}: {get_raster_value(rn, lat, lon)}" for rn in all_rasters.keys()]
            # Actualizamos el texto
            coordinates_label.value = (
                f"Lat: {lat:.6f}, Lon: {lon:.6f}, " + ", ".join(raster_vals)
            )

    the_map.on_interaction(update_coordinates)

    return the_map

# ----------------------------------------------
# 4. Ejecutar: definición de las capas ráster
# ----------------------------------------------

rasters_config = [
    {
        "name": "Acumulacion",
        "path": r"E:\Hidraulica\UD_4\Acumulacion.tif",
        "discrete": True,
        "colormap": "jet"
    },
    {
        "name": "Flooded DEM",
        "path": r"E:\Hidraulica\UD_4\flooded_dem.tif",
        "discrete": False,
        "colormap": "cubehelix"
    },
    {
        "name": "Inflated DEM",
        "path": r"E:\Hidraulica\UD_4\inflated_dem.tif",
        "discrete": False,
        "colormap": "terrain"
    },
    {
        "name": "Distance to Outlet",
        "path": r"E:\Hidraulica\UD_4\distance_to_outlet.tif",
        "discrete": True,
        "colormap": "plasma"
    },
    {
        "name": "Flow Directions",
        "path": r"E:\Hidraulica\UD_4\Direcciones_flujo_recortado.tif",
        "discrete": True,
        "colormap": "jet"
    },
    {
        "name": "MDT",
        "path": r"E:\Hidraulica\UD_3\MDT_recortado.tif",
        "discrete": False,
        "colormap": "terrain"
    }
]

vector_path = "E:/Hidraulica/UD_4/catchment.geojson"
river_network_path = "E:/Hidraulica/UD_4/Red_Drenaje/river_network.geojson"

m = create_interactive_map(vector_path, river_network_path, rasters_config)
m






Map(center=[37.05136602604578, -2.8856154064506194], controls=(ZoomControl(options=['position', 'zoom_in_text'…